# Дообучение YOLOv8 для 8 классов

## Переход с 6 классов на 8 фиксированных классов

## 🎯 8 классов:

Модель обучается на **размеченных данных**, где эксперты уже пометили дефектные объекты:

- **bad_insulator** (3,397 аннотаций) - отсутствующие изоляторы
- **damaged_insulator** (2,063 аннотаций) - поврежденные изоляторы
- **nest** (261 аннотаций) - гнёзда на траверсах 🆕
- **safety_sign** (375 аннотаций) - таблички безопасности 🆕
**Как это работает:**
1. Эксперты просматривают фото и обводят bounding boxes вокруг дефектных объектов
2. Модель учится визуальным признакам дефектов (трещины, отсутствие элементов, сколы)
3. При инференсе модель сравнивает новое изображение с изученными паттернами и определяет, есть ли дефект

**Это supervised learning** - модель учится на примерах, которые уже размечены людьми.


## 1. Установка зависимостей


In [ ]:
!pip install ultralytics -q
!pip install pycocotools -q


## 2. Загрузка датасета

**Вариант 1: Загрузить через Google Drive**
- Подготовьте датасет локально (запустите convert_coco_to_yolo.py и split_dataset.py)
- Заархивируйте папку dataset
- Загрузите в Google Drive
- Смонтируйте Drive здесь


In [ ]:
# Монтирование Google Drive (если датасет там)
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Распаковка архива из Google Drive
# Проверяем несколько возможных путей для архива
import os
from pathlib import Path

dataset_path = Path('/content/dataset_8classes')
possible_archive_paths = [
    Path('/content/drive/MyDrive/Atest/dataset_8classes_for_colab.tar.gz'),
    Path('/content/drive/MyDrive/Colab Notebooks/dataset_8classes_for_colab.tar.gz'),
    Path('/content/drive/MyDrive/dataset_8classes_for_colab.tar.gz'),
]

# Проверяем, не распакован ли уже датасет
if dataset_path.exists() and (dataset_path / 'dataset_8classes.yaml').exists():
    print("✅ Датасет уже распакован!")
    print(f"   Путь: {dataset_path}")
else:
    # Ищем архив в Drive
    print("🔍 Поиск архива dataset_8classes_for_colab.tar.gz в Drive...")
    found_archive = None
    for archive_path in possible_archive_paths:
        print(f"   Проверяем: {archive_path}")
        if archive_path.exists():
            print(f"   ✅ НАЙДЕН: {archive_path}")
            found_archive = archive_path
            break
        else:
            print(f"   ❌ Не найден")
    
    if found_archive:
        print(f"\n📦 Распаковываем архив: {found_archive}")
        os.system(f'tar -xzf "{found_archive}" -C /content/')
        if dataset_path.exists():
            print(f"✅ Датасет успешно распакован в {dataset_path}")
        else:
            print("❌ Ошибка: датасет не распакован!")
    else:
        print("\n⚠️ Архив dataset_8classes_for_colab.tar.gz не найден в Drive!")
        print("   Убедитесь, что архив загружен в одну из папок:")
        for path in possible_archive_paths:
            print(f"   - {path.parent}")


## 3. Проверка структуры датасета


In [ ]:
import os
from pathlib import Path

dataset_path = Path('/content/dataset_8classes')

print("Структура датасета:")
print(f"  Images train: {len(list((dataset_path / 'images' / 'train').glob('*')))} файлов")
print(f"  Images val:   {len(list((dataset_path / 'images' / 'val').glob('*')))} файлов")
print(f"  Images test:  {len(list((dataset_path / 'images' / 'test').glob('*')))} файлов")
print(f"  Labels train: {len(list((dataset_path / 'labels' / 'train').glob('*.txt')))} файлов")
print(f"  Labels val:   {len(list((dataset_path / 'labels' / 'val').glob('*.txt')))} файлов")


## 4. Обновление конфига dataset_8classes.yaml


In [ ]:
import yaml

yaml_path = '/content/dataset_8classes/dataset_8classes.yaml'

# Обновляем путь в конфиге
with open(yaml_path, 'r') as f:
    config = yaml.safe_load(f)

config['path'] = '/content/dataset_8classes'

with open(yaml_path, 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print("Конфиг обновлен:")
print(yaml.dump(config, default_flow_style=False))


## 5. Обучение модели с максимальной точностью

### Параметры для максимальной точности:
- Больше эпох (100-200)
- Больше augmentation
- Больший размер изображения (640 или 1280)
- Использование предобученных весов


In [ ]:
from ultralytics import YOLO
import torch

# Проверка GPU
print(f"CUDA доступна: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Память GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")


## 6. Оценка результатов


## 🔧 Настройка автоматического сохранения чекпоинтов в Drive

**Эта ячейка настраивает автоматическое сохранение чекпоинтов в Google Drive после каждой эпохи.**


In [ ]:
# Функция для автоматического сохранения last.pt в Drive после каждой эпохи
import shutil
from pathlib import Path

# Пути
LAST_PT_SOURCE = Path('/content/runs/detect/yolov8_8classes/weights/last.pt')
DRIVE_BACKUP_DIR = Path('/content/drive/MyDrive/Atest')  # ⚡ Сохраняем в папку Atest
DRIVE_LAST_PT = DRIVE_BACKUP_DIR / 'last.pt'

def save_last_pt_to_drive():
    """Копирует last.pt в Google Drive (вызывается после каждой эпохи)"""
    try:
        if LAST_PT_SOURCE.exists():
            # Создаем папку в Drive
            DRIVE_BACKUP_DIR.mkdir(parents=True, exist_ok=True)
            
            # Копируем только last.pt
            shutil.copy2(LAST_PT_SOURCE, DRIVE_LAST_PT)
            
            size_mb = DRIVE_LAST_PT.stat().st_size / 1024 / 1024
            print(f"💾 last.pt сохранен в Drive! ({size_mb:.1f} MB)")
            return True
        else:
            return False
    except Exception as e:
        print(f"❌ Ошибка при сохранении last.pt в Drive: {e}")
        return False

# Тестовая проверка
print("📋 Настройка автоматического сохранения last.pt в Drive:")
print(f"   Источник: {LAST_PT_SOURCE}")
print(f"   Drive: {DRIVE_LAST_PT}")
DRIVE_BACKUP_DIR.mkdir(parents=True, exist_ok=True)
print(f"   ✅ Папка Drive создана: {DRIVE_BACKUP_DIR.exists()}")


## 📥 Восстановление last.pt из Drive (если есть)

**Выполните эту ячейку ПЕРЕД обучением, чтобы восстановить чекпоинт из Drive.**


In [ ]:
# Шаг 1: Восстановление last.pt из Drive (если есть)
import os
import shutil
import threading
import time
from pathlib import Path
import torch
import torch.serialization
from ultralytics import YOLO
from ultralytics.nn.tasks import DetectionModel

# Разрешаем torch.load загружать YOLO / DetectionModel при weights_only=True
torch.serialization.add_safe_globals([YOLO, DetectionModel])

# ⚡ ПРОВЕРЯЕМ НЕСКОЛЬКИХ ПУТЕЙ для last.pt
possible_drive_paths = [
    Path('/content/drive/MyDrive/Atest/last.pt'),
    Path('/content/drive/MyDrive/Colab Notebooks/last.pt'),
    Path('/content/drive/MyDrive/last.pt'),
]

checkpoint_dir = Path('/content/runs/detect/yolov8_8classes')
last_checkpoint = checkpoint_dir / 'weights' / 'last.pt'
TARGET_EPOCHS = 190  # Целевое количество эпох

# Отладочная информация: проверяем все возможные пути
print("🔍 Поиск last.pt в Drive...")
found_path = None
for path in possible_drive_paths:
    print(f"   Проверяем: {path}")
    if path.exists():
        print(f"   ✅ НАЙДЕН: {path}")
        found_path = path
        break
    else:
        print(f"   ❌ Не найден")

# Восстанавливаем last.pt если найден
EXPECTED_SIZE_MB = 140  # Ожидаемый размер файла (из Drive)
MAX_COPY_ATTEMPTS = 3

if found_path:
    print(f"\n✅ Найден last.pt в Drive: {found_path}")
    print("📥 Восстанавливаем...")
    checkpoint_dir.mkdir(parents=True, exist_ok=True)
    (checkpoint_dir / 'weights').mkdir(parents=True, exist_ok=True)

    copied = False
    for attempt in range(1, MAX_COPY_ATTEMPTS + 1):
        shutil.copy2(found_path, last_checkpoint)
        size_mb = last_checkpoint.stat().st_size / 1024 / 1024
        print(f"   ▸ Попытка {attempt}: {size_mb:.1f} MB")

        if size_mb >= EXPECTED_SIZE_MB:
            copied = True
            print(f"✅ last.pt восстановлен из Drive! ({size_mb:.1f} MB)")
            break
        time.sleep(2)  # Небольшая пауза и повторная попытка

    if not copied:
        raise RuntimeError(
            "last.pt не скопировался полностью. Проверьте файл в Drive и повторите попытку."
        )
else:
    print("\n⚠️ last.pt не найден в Drive. Будет начато обучение с нуля.")

# Функция для обновления epochs внутри самого чекпоинта (чтобы resume действительно шел до TARGET_EPOCHS)
def patch_checkpoint_epochs(checkpoint_path: Path, target_epochs: int):
    try:
        ckpt = torch.load(checkpoint_path, map_location='cpu', weights_only=False)
        updated = False
        for key in ('train_args', 'args'):
            if key in ckpt and ckpt[key] is not None:
                obj = ckpt[key]
                if isinstance(obj, dict):
                    if obj.get('epochs') != target_epochs:
                        obj['epochs'] = target_epochs
                        updated = True
                else:
                    if getattr(obj, 'epochs', None) != target_epochs:
                        setattr(obj, 'epochs', target_epochs)
                        updated = True
        if updated:
            torch.save(ckpt, checkpoint_path)
            print(f"⚙️ Обновили epochs в чекпоинте до {target_epochs}")
        else:
            print(f"⚙️ epochs в чекпоинте уже равен {target_epochs}")
    except Exception as e:
        print(f"⚠️ Не удалось обновить epochs внутри чекпоинта: {e}")

# Шаг 2: Загрузка модели (с чекпоинта или с нуля)
if last_checkpoint.exists():
    patch_checkpoint_epochs(last_checkpoint, TARGET_EPOCHS)
    print(f"\n✅ Найден локальный чекпоинт: {last_checkpoint}")
    print("📌 Продолжаем обучение с последней эпохи...")
    model = YOLO(str(last_checkpoint))
    resume = True
else:
    print("\n🆕 Начинаем обучение с нуля...")
    # Используем yolov8l.pt для максимальной точности
    # (yolov8x.pt - еще лучше, но медленнее; yolov8m.pt - быстрее, но менее точно)
    model = YOLO('yolov8l.pt')  # Large модель для максимальной точности
    resume = False

# Шаг 3: Функции для сохранения чекпоинтов в Drive
# Сохраняем в то же место, откуда был восстановлен, или в Atest по умолчанию
def save_last_pt_to_drive():
    """Копирует last.pt в Google Drive (в то же место, откуда был восстановлен)"""
    try:
        LAST_PT_SOURCE = Path('/content/runs/detect/yolov8_8classes/weights/last.pt')
        
        # Определяем путь для сохранения: используем тот же, откуда восстановили, или Atest по умолчанию
        if found_path:
            DRIVE_LAST_PT = found_path  # Сохраняем туда же, откуда восстановили
        else:
            DRIVE_LAST_PT = Path('/content/drive/MyDrive/Atest/last.pt')  # По умолчанию в Atest
        
        if LAST_PT_SOURCE.exists():
            DRIVE_LAST_PT.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy2(LAST_PT_SOURCE, DRIVE_LAST_PT)
            size_mb = DRIVE_LAST_PT.stat().st_size / 1024 / 1024
            print(f"💾 last.pt сохранен в Drive: {DRIVE_LAST_PT} ({size_mb:.1f} MB)")
            return True
        return False
    except Exception as e:
        print(f"❌ Ошибка при сохранении: {e}")
        return False

def save_best_pt_to_drive():
    """Копирует best.pt в Google Drive (в то же место, где сохраняется last.pt)"""
    try:
        BEST_PT_SOURCE = Path('/content/runs/detect/yolov8_8classes/weights/best.pt')
        
        # Определяем путь для сохранения: используем тот же, откуда восстановили, или Atest по умолчанию
        if found_path:
            DRIVE_BEST_PT = found_path.parent / 'best.pt'  # Сохраняем в ту же папку, где last.pt
        else:
            DRIVE_BEST_PT = Path('/content/drive/MyDrive/Atest/best.pt')  # По умолчанию в Atest
        
        if BEST_PT_SOURCE.exists():
            DRIVE_BEST_PT.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy2(BEST_PT_SOURCE, DRIVE_BEST_PT)
            size_mb = DRIVE_BEST_PT.stat().st_size / 1024 / 1024
            print(f"💾 best.pt сохранен в Drive: {DRIVE_BEST_PT} ({size_mb:.1f} MB)")
            return True
        return False
    except Exception as e:
        print(f"❌ Ошибка при сохранении best.pt: {e}")
        return False

# Шаг 4: Запускаем мониторинг last.pt для автоматического сохранения в Drive
def monitor_last_pt():
    """Мониторит файл last.pt и копирует его в Drive когда он обновляется"""
    last_mtime = 0
    while True:
        try:
            if last_checkpoint.exists():
                current_mtime = last_checkpoint.stat().st_mtime
                # Если файл обновился (новая эпоха завершена)
                if current_mtime > last_mtime:
                    save_last_pt_to_drive()
                    last_mtime = current_mtime
            time.sleep(10)  # Проверяем каждые 10 секунд
        except Exception as e:
            print(f"⚠️ Ошибка в мониторе: {e}")
            time.sleep(30)

# Запускаем мониторинг в фоновом потоке
monitor_thread = threading.Thread(target=monitor_last_pt, daemon=True)
monitor_thread.start()
print("✅ Мониторинг last.pt запущен - будет копироваться в Drive после каждой эпохи")

# Шаг 5: Обучение с сохранением каждой эпохи
print("\n🚀 Начинаем обучение...")
print("💾 last.pt будет автоматически копироваться в Drive после КАЖДОЙ эпохи\n")

results = model.train(
    data='/content/dataset_8classes/dataset_8classes.yaml',
    epochs=190,  # ⚡ УМЕНЬШЕНО с 150 (достаточно для сходимости)
    imgsz=800,   # Размер изображения
    batch=12,    # ⚡ УМЕНЬШЕНО с 32 (чтобы не переполнить RAM)
    device=0,    # GPU 0 (или 'cpu' если нет GPU)
    workers=4,   # ⚡ УМЕНЬШЕНО с 8 (меньше потребление RAM)
    
    # ⚡ КЛЮЧЕВОЙ ПАРАМЕТР для продолжения обучения
    resume=resume,  # True = продолжает с last.pt, False = начинает заново
    
    # 💾 Сохранение - КАЖДУЮ ЭПОХУ
    save=True,          # Сохранять чекпоинты
    save_period=1,      # ⚡ Сохранять КАЖДУЮ эпоху
    
    # ⚡ Оптимизация для лучшей точности
    cache=False,        # ⚡ ОТКЛЮЧЕНО (cache=True переполняет RAM)
    patience=30,        # Early stopping (остановка если нет улучшения 30 эпох)
    
    # Augmentation (оптимизировано)
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    degrees=10, translate=0.1, scale=0.5, shear=5,
    perspective=0.0, flipud=0.0, fliplr=0.5,
    mosaic=0.5,        # ⚡ УМЕНЬШЕНО с 1.0 (меньше агрессивная augmentation)
    mixup=0.05,        # ⚡ УМЕНЬШЕНО с 0.1
    copy_paste=0.05,   # ⚡ УМЕНЬШЕНО с 0.1
    
    # Оптимизация
    optimizer='AdamW', lr0=0.001, lrf=0.01,
    momentum=0.937, weight_decay=0.0005,
    warmup_epochs=3, warmup_momentum=0.8,
    
    # Дополнительные параметры
    val=True,           # Валидация
    plots=True,        # Генерировать графики
    verbose=True,      # Подробный вывод
    
    # Loss weights
    cls=0.5, box=7.5, dfl=1.5,
    
    project='runs/detect',
    name='yolov8_8classes',
    exist_ok=True
)

# Шаг 6: Финальное сохранение в Drive после обучения
print("\n💾 Сохраняем финальные чекпоинты в Drive...")
save_last_pt_to_drive()
save_best_pt_to_drive()
print("✅ Обучение завершено! last.pt и best.pt сохранены в Drive.")


In [ ]:
# Загрузка лучшей модели
best_model_path = 'runs/detect/yolov8_8classes/weights/best.pt'
best_model = YOLO(best_model_path)

# Валидация на тестовом наборе
metrics = best_model.val(data='/content/dataset_8classes/dataset_8classes.yaml', split='test')

print("\n" + "="*60)
print("РЕЗУЛЬТАТЫ ОБУЧЕНИЯ")
print("="*60)
print(f"mAP@0.5: {metrics.box.map50:.4f}")
print(f"mAP@0.5:0.95: {metrics.box.map:.4f}")
print("="*60)

if metrics.box.map50 >= 0.7:
    print("✅ Целевая метрика достигнута! (mAP@0.5 ≥ 0.7)")
if metrics.box.map50 >= 0.85:
    print("🎉 Идеальная метрика достигнута! (mAP@0.5 ≥ 0.85)")


## 7. Сохранение модели


In [ ]:
# Копирование best.pt в Drive для скачивания
# Проверяем несколько возможных путей для сохранения
from pathlib import Path
import shutil

best_model_path = Path('runs/detect/yolov8_8classes/weights/best.pt')

# Определяем путь для сохранения (в ту же папку, где last.pt)
possible_drive_paths = [
    Path('/content/drive/MyDrive/Colab Notebooks/best.pt'),
    Path('/content/drive/MyDrive/Atest/best.pt'),
    Path('/content/drive/MyDrive/best.pt'),
]

# Используем первый существующий путь или создаем новый
drive_best_path = None
for path in possible_drive_paths:
    if path.parent.exists():
        drive_best_path = path
        break

if not drive_best_path:
    # Если ни одна папка не существует, используем Colab Notebooks по умолчанию
    drive_best_path = Path('/content/drive/MyDrive/Colab Notebooks/best.pt')
    drive_best_path.parent.mkdir(parents=True, exist_ok=True)

if best_model_path.exists():
    shutil.copy2(best_model_path, drive_best_path)
    size_mb = drive_best_path.stat().st_size / 1024 / 1024
    print(f"✅ best.pt сохранен в Google Drive: {drive_best_path} ({size_mb:.1f} MB)")
else:
    print(f"❌ Файл {best_model_path} не найден!")
